In [1]:
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from google.cloud import bigquery

def exec_select_query(query):
    """
    Executes the given SQL query using the static Google authentication credentials.

    :param query: The SQL query
    :return: A (pandas) dataframe that contains the results
    """
    # Initialize teh Google BigQuery client. The authentication token should be placed in the working directory in the
    # following path: /resources/google.json
    # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.getcwd(), "resources", "google_bkp.json")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.getcwd(), "google.json")
    client = bigquery.Client()

    # Execute the query and retrieve result data (as pandas dataframe)
    result_df = client.query(query).to_dataframe()

    return result_df

def tracker_on_category():
    result_df = exec_select_query(""" SELECT
                                          browser_id,
                                          category,
                                          COUNTIF(filterlist_China_is_blocked) AS blocked_by_China,
                                          COUNTIF(filterlist_France_is_blocked) AS blocked_by_France,
                                          COUNTIF(filterlist_Germany_is_blocked) AS blocked_by_Germany,
                                          COUNTIF(filterlist_Indian_is_blocked) AS blocked_by_India,
                                          COUNTIF(filterlist_Israel_is_blocked) AS blocked_by_Israel,
                                          COUNTIF(filterlist_Japanese_is_blocked) AS blocked_by_Japan,
                                          COUNTIF(filterlist_Scandinavia_is_blocked) AS blocked_by_Scandinavia,
                                          COUNTIF(filterlist_USA_is_blocked) AS blocked_by_USA,
                                          COUNTIF(filterlist_VAE_is_blocked) AS blocked_by_VAE
                                        FROM
                                          measurement.requests
                                        GROUP BY
                                          browser_id,
                                          category
                                          order by browser_id;
                                        """)

    grouped_df = result_df.groupby("category")

    categories = list(set(result_df['category'].tolist()))

    l = list()
    for category in categories:
        df = grouped_df.get_group(category)
        df = df.drop('browser_id', axis=1)

        l.append({'category': category, 'Blocking_List': 'US', 'count': df['blocked_by_USA'].sum()})
        l.append({'category': category, 'Blocking_List': 'IS', 'count': df['blocked_by_Israel'].sum()})
        l.append({'category': category, 'Blocking_List': 'AE', 'count': df['blocked_by_VAE'].sum()})
        l.append({'category': category, 'Blocking_List': 'IN', 'count': df['blocked_by_India'].sum()})
        l.append({'category': category, 'Blocking_List':'CN','count':df['blocked_by_China'].sum()})
        l.append({'category': category, 'Blocking_List': 'NO', 'count': df['blocked_by_Scandinavia'].sum()})
        l.append({'category': category, 'Blocking_List': 'FR', 'count': df['blocked_by_France'].sum()})
        l.append({'category': category, 'Blocking_List': 'DE', 'count': df['blocked_by_Germany'].sum()})
        l.append({'category': category, 'Blocking_List': 'JP', 'count': df['blocked_by_Japan'].sum()})


    df = pd.DataFrame(l)
    df = df[df.category != 'Unknown']

    # Plot adjustments
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
    matplotlib.rcParams['text.usetex'] = False
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    sb.set(rc={'figure.figsize': (13, 7), "font.size": 18, "axes.titlesize": 18, "axes.labelsize": 18,
               "legend.fontsize": 18, "xtick.labelsize": 18, "ytick.labelsize": 18}, style="white")

    fig, ax1 = plt.subplots()
    p = sns.histplot(df, x='category', hue='Blocking_List', weights='count', multiple='stack', ax=ax1)

    sns.move_legend(p, ncol=1, title="", loc="upper left", bbox_to_anchor=(1, 1))

    plt.xlabel('Categories', weight='bold', fontsize=24, va="bottom")
    plt.ylabel('Number of trackers', weight='bold', fontsize=24)
    plt.xticks(rotation=90, fontsize=20)
    plt.tight_layout()
    plt.show()

tracker_on_category()


NameError: name 'tracker_on_locations' is not defined